Разделение выборки на train и val

# Загрузка библиотек

In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 42

# Создание папок с данными

In [3]:
path_od_data = Path('../data/terminal_object_detection_data/')
path_data = Path(path_od_data / 'model_data')
if not os.path.exists(path_data):
    os.makedirs(path_data)
path_train = Path(path_data / 'train')
if not os.path.exists(path_train):
    os.makedirs(path_train)
    os.makedirs(path_train / 'images')
    os.makedirs(path_train / 'labels')
path_val = Path(path_data / 'val')
if not os.path.exists(path_val):
    os.makedirs(path_val)
    os.makedirs(path_val / 'images')
    os.makedirs(path_val / 'labels')

# Перемещение аугментированных "ancticlass" фотографий

In [4]:
folders_augm = [x for x in os.listdir(path_od_data / 'photos') if 'augm' in x]

In [5]:
for folder in folders_augm:
    path_photo_folder = path_od_data / 'photos' / folder
    for filename in tqdm(os.listdir(path_photo_folder)):
        full_path_photo = path_photo_folder / filename
        full_path_label = full_path_photo.__str__().replace('photos', 'yolo_annot').replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
        shutil.copy(full_path_photo, path_train / 'images' / full_path_photo.__str__().split('\\')[-1])
        shutil.copy(full_path_label, path_train / 'labels' / full_path_label.__str__().split('\\')[-1])

100%|██████████| 3424/3424 [00:03<00:00, 1058.63it/s]


# Разделение исходных фотографий на train и val

In [3]:
val_size = 0.15

In [4]:
df_markup = pd.read_pickle('markup.pkl')

In [5]:
df_markup['flags'].unique()

array(['terminals_augm', 'receipts', 'receipts_augm', 'pc_screens_augm',
       'stickers_augm', 'new_terminals_augm', 'terminals_side_view_augm',
       'other_augm', 'terminals', 'pc_screens', 'rack_new_terminal_augm',
       'new_terminals', 'terminals_side_view', 'other', 'stickers',
       'terminals_receipt', 'rack_new_terminal',
       'wrong_photos_of_terminals'], dtype=object)

In [6]:
df_markup = df_markup.loc[
    df_markup['flags'].isin([
        'new_terminals', 'other', 'pc_screens', 'rack_new_terminal',
       'receipts', 'stickers', 'terminals', 'terminals_side_view'
    ])
]

In [7]:
train, val = train_test_split(df_markup, stratify=df_markup['flags'], test_size=val_size, random_state=RANDOM_STATE)
val.to_pickle('df_val.pkl')

In [11]:
for filename in tqdm(train['filename']):
    shutil.copy(filename, path_train / 'images')
    label_name = filename.replace('photos', 'yolo_annot').replace('.jpg', '.txt').replace('.jpeg', '.txt')
    shutil.copy(label_name, path_train / 'labels')

for filename in tqdm(val['filename']):
    shutil.copy(filename, path_val / 'images')
    label_name = filename.replace('photos', 'yolo_annot').replace('.jpg', '.txt').replace('.jpeg', '.txt')
    shutil.copy(label_name, path_val / 'labels')

100%|██████████| 392/392 [00:00<00:00, 396.16it/s]
